In [72]:
import random
import pickle
import numpy as np
from tqdm import tqdm
import torchfile
import time
import os
# from ..args import get_parser
import time
import lmdb
import shutil
import sys
sys.path.append("..")
from args import get_parser

import utils

ModuleNotFoundError: No module named 'utils'

In [20]:
# Maxim number of images we want to use per recipe
maxNumImgs = 5

def get_st(ids, info):
#     info = torchfile.load(file)
#     print(info.keys())
#     ids = info[b'ids']

    imids = []
    for i,id in enumerate(tqdm(ids)):
        imids.append(''.join(chr(i) for i in id))

    st_vecs = {}
    st_vecs['encs'] = info[b'encs']
    st_vecs['rlens'] = info[b'rlens']
    st_vecs['rbps'] = info[b'rbps']
    st_vecs['ids'] = imids

    # print(np.shape(st_vecs['encs']),len(st_vecs['rlens']),len(st_vecs['rbps']),len(st_vecs['ids']))
    return st_vecs


In [21]:
# don't use this file once dataset is clean
with open('data/remove1M.txt','r') as f:
    remove_ids = {w.rstrip(): i for i, w in enumerate(f)}

In [22]:
info = torchfile.load('data/encs_test_1024.t7')
print(info.keys())
ids = info[b'ids']

dict_keys([b'ids', b'rbps', b'rlens', b'encs', b'opts', b'eor'])


In [23]:
st_vecs_test = get_st(ids, info)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 154045/154045 [00:00<00:00, 369268.64it/s]


In [55]:
import json

with open('data/recipe1M/det_ingrs.json', 'r') as f:
  ingr_data = json.load(f)

In [88]:
ingr_data[1000]

{'valid': [True, True, True, True, True, True, True, True],
 'id': '0040aa3b23',
 'ingredients': [{'text': 'crescent rolls'},
  {'text': 'sausage'},
  {'text': 'swiss cheese'},
  {'text': 'cheddar cheese'},
  {'text': 'eggs'},
  {'text': 'oregano'},
  {'text': 'garlic powder'},
  {'text': 'parmesan cheese'}]}

In [24]:
st_vecs = {'test':st_vecs_test}
stid2idx = {'test':{}}

# for part in ['train','val','test']:
for part in ['test']:
    for i,id in enumerate(st_vecs[part]['ids']):
        stid2idx[part][id] = i

In [89]:
stid2idx['test']['0040aa3b23']

KeyError: '0040aa3b23'

In [69]:
len(stid2idx['test'])

154045

In [70]:
stid2idx['test']

{'00003a70b1': 0,
 '0001bdeec0': 1,
 '0002491373': 2,
 '00047059be': 3,
 '0004dbbc43': 4,
 '00059477e2': 5,
 '0005bdba14': 6,
 '0006604a20': 7,
 '0007013e7e': 8,
 '000731b459': 9,
 '0007a28fe7': 10,
 '00093c7461': 11,
 '0009785a2e': 12,
 '0009dae0d8': 13,
 '000a102b26': 14,
 '000a3d10fb': 15,
 '000bba053c': 16,
 '000d4a69f9': 17,
 '000d5e4996': 18,
 '000d94f80b': 19,
 '000e080daa': 20,
 '000f0026ea': 21,
 '000f2b5cf0': 22,
 '000f46b29f': 23,
 '000fc78329': 24,
 '000ff73a93': 25,
 '00100336d5': 26,
 '00100a0aa5': 27,
 '0010cfb6ca': 28,
 '001243534a': 29,
 '0012733d1d': 30,
 '0012806b36': 31,
 '001310c451': 32,
 '0013592bfb': 33,
 '001444c62a': 34,
 '0014d60e8b': 35,
 '001528065f': 36,
 '0015a6e965': 37,
 '0015f3dbc0': 38,
 '001628a33d': 39,
 '00164335fc': 40,
 '0016990a89': 41,
 '0016a83971': 42,
 '0016c315f5': 43,
 '0016ca65dd': 44,
 '0017742044': 45,
 '001788bd82': 46,
 '0017898fda': 47,
 '0017bcd23f': 48,
 '0017c79fb6': 49,
 '00181ba854': 50,
 '00183acce9': 51,
 '0018949bd3': 52,
 '0

In [30]:
st_vecs_test

{'encs': array([[ 5.74587600e-12, -1.79933707e-07, -4.58599950e-17, ...,
          6.45879034e-17,  1.30247368e-04,  4.63572695e-21],
        [ 1.37136269e-07, -6.20312823e-09, -9.85455915e-18, ...,
          3.09212289e-17,  1.26524093e-02,  2.56147802e-20],
        [ 1.28299851e-08, -6.23688811e-06, -1.86991134e-12, ...,
         -2.69421143e-16,  2.89116055e-03,  2.78444558e-16],
        ...,
        [ 5.51057810e-10,  7.09476171e-06, -1.20557030e-21, ...,
          4.81650716e-14,  7.90870376e-03,  1.68672719e-21],
        [ 2.38474546e-10, -4.89219837e-03, -1.05293804e-19, ...,
          1.26250441e-15,  2.28522708e-06,  8.02630409e-20],
        [ 1.58526658e-09, -2.45663594e-03, -6.07057307e-23, ...,
          1.94287420e-15,  1.65416554e-04, -5.16904382e-24]], dtype=float32),
 'rlens': array([ 9, 21, 28, ...,  6,  7, 11]),
 'rbps': array([      1,      10,      31, ..., 1605253, 1605259, 1605266]),
 'ids': ['00003a70b1',
  '0001bdeec0',
  '0002491373',
  '00047059be',
  '0004dbb

In [35]:
len(st_vecs_test['encs']), len(st_vecs_test['rlens']), len(st_vecs_test['rbps']), len(st_vecs_test['ids'])

(1605276, 154045, 154045, 154045)

In [91]:
st_vecs_test['encs'].shape

(1605276, 1024)

In [47]:
st_vecs_test['encs'][0].shape

(1024,)

In [48]:
st_vecs_test['rlens'][0]

9

In [50]:
st_vecs_test['rbps'][0]

1

In [52]:
st_vecs_test['ids'][0]

'00003a70b1'

In [74]:
%ls

Dockerfile*   args.py*             requirements.txt*  test.py*
LICENSE*      data/                results/           test_vec.ipynb*
README.md*    data_loader.py*      scripts/           train.py*
__pycache__/  image2embedding.py*  snapshots/         trijoint.py*


In [77]:
import sys  
sys.path.insert(0, 'scripts/')

import utils

In [79]:
DATASET = 'data/recipe1M/'

dataset = utils.Layer.merge([utils.Layer.L1, utils.Layer.L2, utils.Layer.INGRS],DATASET)
# print('Loading ingr vocab.')
with open(opts.vocab) as f_vocab:
    ingr_vocab = {w.rstrip(): i+2 for i, w in enumerate(f_vocab)} # +1 for lua
    ingr_vocab['</i>'] = 1

with open('classes1M.pkl','rb') ars f:
    class_dict = pickle.load(f)
    id2class = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/recipe1M/layer2.json'

In [83]:
with (open("data/test_keys.pkl", "rb")) as openfile:
    while True:
        try:
            objects = pickle.load(openfile)
        except EOFError:
            break

In [84]:
objects

['00003a70b1',
 '00047059be',
 '00059477e2',
 '0007a28fe7',
 '000bba053c',
 '000d5e4996',
 '00100336d5',
 '001243534a',
 '0012733d1d',
 '0012806b36',
 '0013592bfb',
 '001528065f',
 '0015a6e965',
 '0015f3dbc0',
 '0017c79fb6',
 '00183acce9',
 '0018949bd3',
 '001ac13db4',
 '001ee1021c',
 '001fc91fde',
 '00202f23e5',
 '002152a50f',
 '0022b163f8',
 '00234a6bd1',
 '0024f517bc',
 '0025ca4eef',
 '0027c75380',
 '0027f74f8c',
 '00285184ba',
 '00288dc007',
 '0028f57fc0',
 '0029fb3ebf',
 '002a1c460f',
 '002bac0f54',
 '002cc78992',
 '002dd9056e',
 '002e6cc0eb',
 '002fd94c16',
 '003045288d',
 '003642bf69',
 '003868aa58',
 '0039542254',
 '003991406d',
 '0039c547a7',
 '003c17c817',
 '003e231a46',
 '003e8990ab',
 '003ef1cc1e',
 '003f212b7c',
 '004060114d',
 '0040c511dd',
 '00421bfcd3',
 '00439a5d01',
 '004628ff8d',
 '0046365abb',
 '0049a8b6dc',
 '004ac39841',
 '004ce368d7',
 '004d4dbfb0',
 '004dec22f2',
 '0050eb2f27',
 '00514430c3',
 '00594aae85',
 '005a7aa8db',
 '005ad8ac22',
 '005c236d80',
 '005f5336